In [10]:
import os
import pandas as pd
# the utils.py file contains the reusable functions from the other notebooks
import utils as ut
from utils import cleanup_abstract
full_data_folder = '/Users/junyi/Work/RR/rr-measure-dataset/journal-full-text'
meta_data_folder = '/Users/junyi/Work/RR/rr-measure-dataset/journal-meta'

In [12]:
# List all files in the folder with csv
journals = [f for f in os.listdir(meta_data_folder) if f.endswith('.csv')]
journal_issn_list = [['TRA','0965-8564'],
                     ['TRB','0191-2615'],
                     ['TRC','0968-090X'],
                     ['TRD','1361-9209'],
                     ['TRE','1366-5545'],
                     ['TRF','1369-8478'],
                     ['TRIP','2590-1982']]
journal_issn_list_df = pd.DataFrame(journal_issn_list,columns=['journal_name','issn'])
all_data = pd.DataFrame()
for journal in journals:
    # skip if the journal is not in the list
    if journal.split('.')[0] not in journal_issn_list_df['issn'].values:
        continue
    # Get the journal name
    journal_name = journal_issn_list_df[journal_issn_list_df['issn']==journal.split('.')[0]]['journal_name'].values[0]
    data = pd.read_csv(os.path.join(meta_data_folder,journal))
    # Clean up the journal abstract
    data['abstract'] = data['abstract'].apply(str)
    data['abstract'] = data['abstract'].apply(cleanup_abstract)
    data['issn'] = journal.split('.')[0]
    data['journal_name'] = journal_name
    all_data = pd.concat([all_data,data],axis=0)

In [13]:
all_data.reset_index(drop=True,inplace=True)
all_data['unique_id'] = all_data['doi'].apply(ut.doi_to_unique_id)

In [14]:
all_data.head()

,title,doi,volume,date,year,month,abstract,issn,journal_name,unique_id
0,Decentralized network level adaptive signal co...,10.1016/j.trip.2019.100020,1,2019-06-01,2019,6,Adaptive traffic signal control systems are de...,2590-1982,TRIP,10.1016_j.trip.2019.100020
1,Physical activity of electric bicycle users co...,10.1016/j.trip.2019.100017,1,2019-06-01,2019,6,Physical activity has been widely associated w...,2590-1982,TRIP,10.1016_j.trip.2019.100017
2,Increasing civil aviation capacity in China re...,10.1016/j.trip.2019.100005,1,2019-06-01,2019,6,China is the world's second largest aviation m...,2590-1982,TRIP,10.1016_j.trip.2019.100005
3,Progress or regress on gender equality: The ca...,10.1016/j.trip.2019.100009,1,2019-06-01,2019,6,This paper examines the role of vocational edu...,2590-1982,TRIP,10.1016_j.trip.2019.100009
4,Multiobjective integrated signal-control syste...,10.1016/j.trip.2019.100011,1,2019-06-01,2019,6,"Parameters concerning real-time, advanced traf...",2590-1982,TRIP,10.1016_j.trip.2019.100011


In [16]:
if not os.path.exists(meta_data_folder):
    os.makedirs(meta_data_folder)
all_data.to_csv(meta_data_folder+'/full-meta-dataset.csv',index=False)